<a href="https://colab.research.google.com/github/praxxley-tech/crypto/blob/main/coinBras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Api von Coingecko als Grundlage:

In [ ]:
pip install pycoingecko

In [ ]:
from pycoingecko import CoinGeckoAPI
import datetime
import requests
from bs4 import BeautifulSoup

cg = CoinGeckoAPI()

def get_crypto_data():
    top_100_coins = cg.get_coins_markets(vs_currency='usd', per_page=100)

    end_date = datetime.date.today() - datetime.timedelta(days=180)

    for coin in top_100_coins:
        coin_id = coin['id']
        historical_data = cg.get_coin_market_chart_by_id(coin_id, vs_currency='usd', days=180)
        
        # Analyse der historischen Daten und Berechnung von Gewinn/Verlust
        def analyze_historical_data(historical_data):
    prices = historical_data['prices']
    if len(prices) < 2:
        return None

    first_price = prices[0][1]
    last_price = prices[-1][1]

    profit_loss = ((last_price - first_price) / first_price) * 100
    return profit_loss

def get_crypto_data():
    top_100_coins = cg.get_coins_markets(vs_currency='usd', per_page=100)

    end_date = datetime.date.today() - datetime.timedelta(days=180)

    coins_profit_loss = []

    for coin in top_100_coins:
        coin_id = coin['id']
        historical_data = cg.get_coin_market_chart_by_id(coin_id, vs_currency='usd', days=180)

        profit_loss = analyze_historical_data(historical_data)
        if profit_loss is not None:
            coins_profit_loss.append((coin['symbol'], profit_loss))

    coins_profit_loss.sort(key=lambda x: x[1], reverse=True)

    for coin_symbol, profit_loss in coins_profit_loss:
        print(f"{coin_symbol.upper()}: {profit_loss:.2f}%")

if __name__ == '__main__':
    get_crypto_data()


    # Berücksichtigen Sie verbleibende Münzen und Kaufverhalten
    # ...

    # Häufigkeit der Erwähnung in den Nachrichten
    # ...

def get_news_mentions(coin_name):
    url = f'https://news.google.com/search?q={coin_name}%20cryptocurrency&hl=en-US&gl=US&ceid=US:en'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return len(soup.find_all('article'))

if __name__ == '__main__':
    get_crypto_data()
